# Notebook to create 2D initial conditions to Mandyoc

In [7]:
# from functions.coordinates import grid_coordinates, create_grid
# from functions.interfaces import create_interface, interface_from_vertices, merge_interfaces
# from functions.io.interfaces import save_interfaces
# from functions.io.temperature import save_temperature

import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from functions.coordinates import grid_coordinates
from functions.interfaces import create_interface

Create a directory to save the input files:

In [8]:
run_dir = os.path.join("input")
if not os.path.isdir(run_dir):
    os.makedirs(run_dir)

Define and create the mesh geometry:

In [9]:
xmin, xmax, zmin, zmax = 0, 11600e3, -2900e3, 0.0e3  # m
region = (xmin, xmax, zmin, zmax)
shape = (1001, 601)

# Create the coordinates for the region
coordinates = grid_coordinates(region, shape)
coordinates

Coordinates:
  * x        (x) float64 0.0 1.16e+04 2.32e+04 ... 1.158e+07 1.159e+07 1.16e+07
  * z        (z) float64 -2.9e+06 -2.895e+06 -2.89e+06 ... -4.833e+03 0.0

## Interfaces

In [10]:
# Lower boundary of the astenosphere [m]
lower_boundary_astenosphere = -660e3

# Non-subduction oceanic plate [m]
oceanic_lab_depth = -80e3  
oceanic_crust_depth = -10e3 
oceanic_thin_length = 200e3  

# Subducting slab portion [m]
slab_slope = 29  # °
slab_length = 200e3  # Length of subducted slab
slab_xmax = 5700e3  

slab_slope_rad = slab_slope * np.pi / 180 
slab_zmax = - slab_length * np.sin(slab_slope_rad)   # Maximum depth reached by the slab (~ 100 km)  
slab_xmin = slab_xmax - np.cos(slab_slope_rad) * slab_length  # x position where the slab begins to subduct 

# Continental arc region [m]
arc_length = 1300e3 
arc_depth = -60e3

# Non-arc continental region [m]
# Cratoic region
continental_lab_depth = -150e3
continental_upper_crust_depth = -30e3
craton_thin_length = 1600e3

# Craton transition
transition_xmin = slab_xmin + arc_length
transition_xlength = 200e3
transition_xmax = transition_xmin + transition_xlength
a = (continental_lab_depth - arc_depth) / transition_xlength
b = arc_depth - a * transition_xmin
transition_xmidle = (slab_zmax - b) / a

**Lower mantle**

In [12]:
lower_mantle = create_interface(
    coordinates, 
    fill_value=lower_boundary_astenosphere
)
lower_mantle

<xarray.DataArray (x: 1001)>
array([-660000., -660000., -660000., ..., -660000., -660000., -660000.])
Coordinates:
  * x        (x) float64 0.0 1.16e+04 2.32e+04 ... 1.158e+07 1.159e+07 1.16e+07

**Upper mantle**

In [ ]:
upper_mantle_vertices = [
    [xmin, zmax],
    [xmin + oceanic_thin_length, oceanic_lab_depth],
    [slab_xmin, oceanic_lab_depth],
    [slab_xmax, oceanic_lab_depth + slab_zmax],
    [slab_xmax, slab_zmax],
    [transition_xmidle, slab_zmax],
    [transition_xmax, continental_lab_depth],
    [xmax - craton_thin_length, continental_lab_depth],
    [xmax, zmax],
]

upper_mantle = interface_from_vertices(
    upper_mantle_vertices,
    coordinates
)

**Oceanic litho**

In [ ]:
oceanic_litho = upper_mantle.copy() + abs(oceanic_lab_depth - oceanic_crust_depth)

oceanic_litho = xr.where(
    oceanic_litho.x >= slab_xmax,
    upper_mantle,
    oceanic_litho
)

oceanic_litho = xr.where(
    oceanic_litho.x <= xmin + oceanic_thin_length,
    oceanic_crust_depth,
    oceanic_litho
)

oceanic_litho = xr.where(
    (oceanic_litho <= upper_mantle) & (oceanic_litho.x <= xmin + oceanic_thin_length),
    upper_mantle,
    oceanic_litho
)

**Oceanic crust**

In [ ]:
oceanic_crust = oceanic_litho.copy() + abs(oceanic_crust_depth)

oceanic_crust = xr.where(
    oceanic_crust.x >= slab_xmax,
    oceanic_litho,
    oceanic_crust
)

oceanic_crust = xr.where(
    oceanic_crust.x <= xmin + oceanic_thin_length,
    zmax,
    oceanic_crust
)

**Upper mantle 2**

In [ ]:
slab_xmidle = slab_xmin +  arc_depth / np.tan(slab_slope_rad)

upper_mantle_2_vertices = [
    [xmin, arc_depth],
    [slab_xmin, arc_depth],
    [transition_xmin, arc_depth],
    [transition_xmax, continental_lab_depth],
    [xmax, continental_lab_depth],
]

upper_mantle_2 = interface_from_vertices(
    upper_mantle_2_vertices, 
    coordinates
)

upper_mantle_2 = xr.where(
    upper_mantle_2 <= oceanic_crust,
    oceanic_crust,
    upper_mantle_2
)

**Upper plate lithosphere**

In [ ]:
upper_plate_litho_vertices = [
    [xmin, continental_upper_crust_depth],
    [xmax, continental_upper_crust_depth]
]

upper_plate_litho = interface_from_vertices(
    upper_plate_litho_vertices, 
    coordinates
)

upper_plate_litho = xr.where(
    upper_plate_litho <= oceanic_crust,
    oceanic_crust,
    upper_plate_litho
)

Merge all interfaces in a xr.Datarray

In [ ]:
interfaces = merge_interfaces(
    {
        "lower_mantle": lower_mantle,
        "upper_mantle": upper_mantle,
        "oceanic_litho": oceanic_litho,
        "oceanic_crust": oceanic_crust,
        "upper_mantle_2": upper_mantle_2,
        "upper_plate_litho": upper_plate_litho,
    }
)
print(interfaces)

**Plot the interfaces**

In [ ]:
len(interfaces.keys())


In [ ]:
fig, ax = plt.subplots(
    figsize=(15,20)
)

# Change the orientation of the color bar
cbar_kwargs = {"orientation": "horizontal"}
kwargs = {"cbar_kwargs": cbar_kwargs}

# Tnterfaces
for name, interface in interfaces.items():
    interface.plot.line(ax=ax, label=name)
plt.legend(loc="lower right", fontsize="x-small")
ax.ticklabel_format(axis="both", style="sci", scilimits=(0, 0))
ax.set_aspect("equal")
#plt.xlim((5000e3, 8000e3))
#plt.ylim((zmin, 100e3))
plt.ylabel("z")

plt.show()


In [ ]:
fig, ax = plt.subplots(
    figsize=(15,20)
)

# Change the orientation of the color bar
cbar_kwargs = {"orientation": "horizontal"}
kwargs = {"cbar_kwargs": cbar_kwargs}

# Tnterfaces
for name, interface in interfaces.items():
    interface.plot.line(ax=ax, label=name)
plt.legend(loc="lower right", fontsize="x-small")
ax.ticklabel_format(axis="both", style="sci", scilimits=(0, 0))
ax.set_aspect("equal")
plt.xlim((5000e3, 7500e3))

plt.ylim((-250e3, 100))
plt.ylabel("z")

plt.show()

**Save the interface**

In [ ]:
# Radiogenic heat for upper and lower crust:
H_upper_crust = 2.5E-6/2700.0 #4.63e-10 W/kg
H_lower_crust = 0.8E-6/2800.0# 7.14e-11 W/kg

layers_data = {
    # lower_mantle, 
    # upper_mantle,
    # oceanic_litho, 
    # oceanic_crust, 
    # upper_mantle2,
    # upper_plate_litho, 
    # upper_plate_crust
    "compositional_factor": [1, 1, 100, 0.01, 1, 1, 1,],
    "density": [3378., 3378., 3354., 2800., 3378., 3354., 2700.,],
    "radiogenic_heat": [0.0, 0.0, 9.0e-12, H_lower_crust, 0.0, 9.0e-12, H_upper_crust,],
    "pre-exponential_scale_factor": [1.393e-14,
                                     1.393e-14, 
                                     2.4168e-15, 
                                     8.574e-28,
                                     1.393E-14,
                                     2.4168E-15,
                                     8.574e-28,
                                    ],
    "power_law_exponent": [3., 3., 3.5, 4., 3., 3.5, 4.,],
    "activation_energy": [429.0E3, 429.0E3, 540.0E3, 222.0E3, 429.0E3, 540.0E3, 222.0E3,],
    "activation_volume": [15.0E-6,
                          15.0E-6,
                          25.0E-6,
                          0,
                          15.0E-6, 
                          25.0E-6,
                          0,
                         ],
}

# Save data
save_interfaces(interfaces, layers_data, run_dir)


## Initial temperature model

Create the temperature dataset

In [ ]:
temperature = create_grid(coordinates, fill_value=0)
temperature

Create the lithospere interface without the subducted slab to calculate the lithospheric temperature:

In [ ]:
# Create the lithospheric interface by interpolating the vertices:
lid_vertices = [
    [xmin, zmax],
    [xmin + oceanic_thin_length, oceanic_lab_depth],
    [slab_xmin, oceanic_lab_depth],
    [slab_xmax, arc_depth],
    [transition_xmin, arc_depth],
    [transition_xmax, continental_lab_depth],
    [xmax - craton_thin_length, continental_lab_depth],
    [xmax, zmax]
]

lid = interface_from_vertices(lid_vertices, coordinates)

The lithospheric temperature increases linearly with depth as:
$$ T(z) = -\frac{(1330°C) z}{thickness_{litho}} $$

In the asthenosphere, the temperature is the adiabatic temperature.

In [ ]:
temp_moho = 1300  # °
heat_capacity = 1250  # Cp m2/Ks2 
alpha = 3.28E-5  # thermal expansion coefficient 1/K 
g = 9.8  # gravity m/s2
temp_0 = 1262  # °
particles_per_element = 30
kappa = 1.0e-6

In [ ]:
# Add the linear temperature of the lithosphere in the dataset:
temperature = temp_moho * (-temperature.z) / abs(lid)

# Add the adiabatic temperature in the dataset:
asthenosphere_temperature =  temp_0 / np.exp(-g * alpha * (-temperature.z) / heat_capacity)

temperature = xr.where(
    temperature > asthenosphere_temperature,
    asthenosphere_temperature, 
    temperature
)

# Transpoce the temperature to fix the dimension
temperature = temperature.transpose("x", "z")

# Replaces the upper extremes values from nan to 0. 
temperature = xr.where(
    (temperature.x==xmax) & (temperature.z==zmax),
    0, 
    temperature
) 
temperature = xr.where(
    (temperature.x==xmin) & (temperature.z==zmax),
    0, 
    temperature
) 
# Change -0.0 by 0.0
temperature = xr.where(
    temperature==-0,
    0.0, 
    temperature
) 

**Calculate the temperature of the subducted slab**

First it necessary to calculate the top and botom boundary of the slab:

In [ ]:
# Calculate the top boundary of the subducted slab:
top = -np.tan(slab_slope_rad) * (temperature.x - slab_xmin)

# Calculate the bottom boundary of the subducted slab:
bottom = top + oceanic_lab_depth

Add the temperature of the subducted slab in the dataset:

In [ ]:
# Condition to generate the slab temperature
condition = (
    (temperature.x > slab_xmin)
    & (temperature.x < slab_xmax)
    & (temperature.z < top)
    & (temperature.z > bottom)
)

# Temperature in the subducted slab
temp_aux = (temperature.data + 1300 * (top - temperature.z) / abs(oceanic_lab_depth)) / 2

# Add this in the temperature dataset
temperature.data[condition] = temp_aux.data[condition]
temperature

**Plot the temperature**

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
# Change the orientation of the color bar
cbar_kwargs = {"orientation": "horizontal"}
kwargs = {"cbar_kwargs": cbar_kwargs}

temperature.plot(y="z", **kwargs)
ax.ticklabel_format(axis="both", style="sci", scilimits=(0, 0))
ax.set_aspect("equal")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
# Change the orientation of the color bar
cbar_kwargs = {"orientation": "horizontal"}
kwargs = {"cbar_kwargs": cbar_kwargs}

temperature.plot(y="z", **kwargs)
ax.ticklabel_format(axis="both", style="sci", scilimits=(0, 0))
ax.set_aspect("equal")

ax.set_yticks(
    (oceanic_crust_depth, oceanic_lab_depth, arc_depth, continental_lab_depth))

plt.xlim((5000e3, 7500e3))
plt.ylim((-300e3, zmax))

#plt.plot(top.x, top.data)
#plt.plot(bottom.x, bottom.data)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))

# temperature in the oceanic crust
plt.subplot(131)
aux = temperature.sel(x=5000e3, method="nearest") 
aux.plot(y="z")
plt.title("Oceanic plate {} km".format(aux.x.values / 1000))
plt.plot(
    aux, 
    oceanic_lab_depth * np.ones_like(aux.z),
    label="oceanic_lithosphere ({} km)".format(oceanic_lab_depth / 1e3)
)
plt.ylim((-300e3, zmax))
plt.legend()

# temperature in the arc zone
plt.subplot(132)
aux = temperature.sel(x=6500e3, method="nearest") 
aux.plot(y="z")
plt.title("Oceanic plate {} km".format(aux.x.values / 1000))
plt.plot(
    aux, 
    arc_depth * np.ones_like(aux.z),
    label="oceanic_lithosphere ({} km)".format(arc_depth / 1e3)
)
plt.ylim((-300e3, zmax))
plt.legend()

# temperature in the craton area
plt.subplot(133)
aux = temperature.sel(x=7500e3, method="nearest") 
aux.plot(y="z")
plt.title("Oceanic plate {} km".format(aux.x.values / 1000))
plt.plot(
    aux, 
    continental_lab_depth * np.ones_like(aux.z),
    label="oceanic_lithosphere ({} km)".format(continental_lab_depth / 1e3)
)
plt.ylim((-300e3, zmax))
plt.legend()

plt.show()

**Save initial temperature distribution**

In [ ]:
save_temperature(temperature, run_dir)

### Create the parameter file

In [ ]:
# params = f"""
# # Geometry
# nx                                  = {shape[0]}           # Number of elements in the longitudinal direction
# nz                                  = {shape[1]}           # Number of elements in the vertical direction
# lx                                  = {xmax}               # Extent in the longitudinal direction
# lz                                  = {-zmin}              # Extent in the vertical direction
# # Simulation options
# multigrid                           = 1
# solver                              = direct        # default is direct [direct/iterative]
# denok                               = 1.0e-15       # default is 1.0E-4
# particles_per_element               = 30           # default is 81
# particles_per_element_x             = 0            # Number of particles per element in longitudinal (default is 0)
# particles_per_element_z             = 0           # Number of particles per element in vertical (default is 0)
# particles_perturb_factor            = 0.7           # default is 0.5 [values are between 0 and 1]
# rtol                                = 1.0e-7        # the absolute size of the residual norm (relevant only for iterative methods), default is 1.0E-5
# RK4                                 = Euler         # default is Euler [Euler/Runge-Kutta]
# Xi_min                              = 1.0e-14       # default is 1.0E-14
# random_initial_strain               = 0.0           # default is 0.0
# pressure_const                      = -1.0          # default is -1.0 (not used) - useful only in horizontal 2D models
# initial_dynamic_range               = False         # default is False [True/False]
# periodic_boundary                   = False         # default is False [True/False]
# high_kappa_in_asthenosphere         = False         # default is False [True/False]
# K_fluvial                           = 2.0e-7        # default is 2.0E-7
# m_fluvial                           = 1.0           # default is 1.0
# sea_level                           = 0.0           # default is 0.0
# basal_heat                          = -1.0          # default is -1.0
# # Surface processes
# sp_surface_tracking                 = False         # default is False [True/False]
# sp_surface_processes                = False         # default is False [True/False]
# sp_mode                             = 0             # default is 1 [0/1/2]
# sp_dt                               = 0             # default is 0.0
# sp_d_c                              = 0.0           # default is 0.0
# plot_sediment                       = False         # default is False [True/False]
# a2l                                 = True          # default is True [True/False]
# free_surface_stab                   = True          # default is True [True/False]
# theta_FSSA                          = 0.5           # default is 0.5 (only relevant when free_surface_stab = True)
# # Time constrains
# step_max                            = 7000          # Maximum time-step of the simulation
# time_max                            = 1.0e6         # Maximum time of the simulation [years]
# dt_max                              = 10.0e3        # Maximum time between steps of the simulation [years]
# step_print                          = 10            # Make file every <step_print>
# sub_division_time_step              = 1.0           # default is 1.0
# initial_print_step                  = 2             # default is 0
# initial_print_max_time              = 1.0e6         # default is 1.0e6 [years]
# # Viscosity
# viscosity_reference                 = 3.5e20        # Reference viscosity [Pa.s] !! check this value in the future
# viscosity_max                       = 1.0e23        # Maximum viscosity [Pa.s] !! check this value in the future
# viscosity_min                       = 3.5e19        # Minimum viscosity [Pa.s] !! check this value in the future
# viscosity_per_element               = constant      # default is variable [constant/variable]
# viscosity_mean_method               = arithmetic    # default is harmonic [harmonic/arithmetic]
# viscosity_dependence                = pressure      # default is depth [pressure/depth]
# # External ASCII inputs/outputs
# interfaces_from_ascii               = True          # default is False [True/False]
# n_interfaces                        = {len(interfaces.keys())}  # Number of interfaces in the 'interfaces.txt' file
# variable_bcv                        = False         # default is False [True/False]
# temperature_from_ascii              = True          # default is False [True/False]
# velocity_from_ascii                 = False         # default is False [True/False]
# binary_output                       = False         # default is False [True/False]
# sticky_blanket_air                  = False         # default is False [True/False]
# precipitation_profile_from_ascii    = False         # default is False [True/False]
# climate_change_from_ascii           = False         # default is False [True/False]
# print_step_files                    = True          # default is True [True/False]
# checkered                           = False         # Print one element in the print_step_files (default is False [True/False])
# geoq                                = on
# geoq_fac                            = 1.0
# # Physical parameters
# temperature_difference              = {temperature.values.max() - temperature.values.min()}
# thermal_expansion_coefficient       = {alpha}
# thermal_diffusivity_coefficient     = 
# gravity_acceleration                = {g}
# density_mantle                      = 3230.0
# external_heat                       = 0.0e-12
# heat_capacity                       = {heat_capacity}   # Check smoller values for the future
# non_linear_method                   = on
# adiabatic_component                 = off
# radiogenic_component                = on
# # Velocity boundary conditions
# top_normal_velocity                 = fixed
# top_tangential_velocity             = free
# bot_normal_velocity                 = fixed
# bot_tangential_velocity             = free
# left_normal_velocity                = fixed
# left_tangential_velocity            = free
# right_normal_velocity               = fixed
# right_tangential_velocity           = free
# surface_velocity                    = 0.0
# multi_velocity                      = False         # default is False [True/False]
# # Temperature boundary conditions
# top_temperature                     = fixed
# bot_temperature                     = fixed
# left_temperature                    = free
# right_temperature                   = free
# rheology_model                      = 19
# T_initial                           = 0
# """

# # Create the parameter file
# with open(os.path.join(run_dir,"param.txt"), "w") as f:
#     for line in params.split("\n"):
#         f.write(line + "\n")

In [ ]:
params = f"""# Geometry
nx                                  = {shape[0]}            # n. of nodes in the horizontal direction
nz                                  = {shape[1]}            # n. of nodes in the vertical direction
lx                                  = {xmax}       # extent in the horizontal direction
lz                                  = {-zmin}       # extent in the vertical direction 

# Simulation options
solver                              = direct        # set the solver to be direct or iterative, default is direct [direct/iterative]
denok                               = 1.0e-10       # tolerance criterion for the Uzawa's scheme, default is 1.0e-4
rtol                                = 1.0e-5        # the absolute size of the residual norm (relevant only for iterative methods), default is 1.0e-5
RK4                                 = Euler         # particles advection method, default is Euler [Euler/Runge-Kutta]
Xi_min                              = 1.0e-10       # tolerance criterion for the convergence of the non-linear flow, default is 1.0e-14
random_initial_strain               = 0.0           # non-dimensional value for the initial strain perturbation for the entire domain, default is 0.0 
pressure_const                      = -1.0          # set constant pressure value for the domain (relevant only when 2D is plain view), default is -1.0 (i.e. not used) [Pa]
initial_dynamic_range               = False         # method to smoothen convergence of the velocity field in scenarios with wide viscosity range, default is False [True/False], Gerya (2019)
periodic_boundary                   = False         # allows simulation with periodic boundary in the horizontal direction, default is False [True/False]
high_kappa_in_asthenosphere         = False         # mimics high heat transport in the asthenosphere increasing its thermal diffusivity coefficient, default is False [True/False]
basal_heat                          = -1.0          # set basal heat flux value, default is -1.0 (i.e. not used) [W/m^2]

# Particles options
particles_per_element               = {particles_per_element}          # n. of Lagrangian particles in each element, default is 81
particles_per_element_x             = 0             # n. of Lagrangian particles in the horizontal direction, default is 0 for automatic calculation
particles_per_element_z             = 0             # n. of Lagrangian particles in the vertical direction, default is 0 for automatic calculation
particles_perturb_factor            = 0.0           # indicates the amount of perturbation of the initial location of the particles relative to a regular grid distribution. Default is 0.5 [values are between 0 and 1]

# Surface processes
sp_surface_tracking                 = False         # allows free surface tracking across time and outputs it, default is False [True/False]
sea_level                           = 0.0           # sea level used to limit the influence of the surface process, default is 0.0 [m]
sp_surface_processes                = False         # allows erosion and sedimentation simulation, default is False [True/False]
sp_dt                               = 0             # time step for surface processes simulation, default is 0 [years]
a2l                                 = True          # allows the conversion of air particles to land particles during sedimentation, default is True [True/False]
sp_mode                             = 1             # specify the surface processes method, default is 1
free_surface_stab                   = True          # set if the free surface stabilization algorithm (FSSA) is used, default is True [True/False], Kaus et al. (2010)
theta_FSSA                          = 0.5           # weight of the influence of the FSSA method, default is 0.5 (only relevant when <free_surface_stab> is True)
sticky_blanket_air                  = False         # allows the increase of viscosity for the first air layer of particles, default is False [True/False]
precipitation_profile_from_ascii    = False         # set if precipitation profile along the horizontal axis is read from an ASCII file, default is False [True/False]
climate_change_from_ascii           = False         # if True, re-scales through time the precipitation profile using an ASCII file, default is False [True/False]

# Time constrains
step_max                            = 7000          # maximum time-step of the simulation [steps]
time_max                            = 10.0e6        # maximum time of the simulation [years]
dt_max                              = 10.0e3        # maximum time between steps of the simulation [years]
step_print                          = 10            # make output files every <step_print>
sub_division_time_step              = 1.0           # re-scale value for the calculated time-step, default is 1.0
initial_print_step                  = 0             # <step_print> used until <initial_print_max_time>, default is 0 (i.e. not used)
initial_print_max_time              = 1.0e6         # maximum time to make output files every <initial_print_step>, default is 1.0e6 [years]

# Viscosity
viscosity_reference                 = 3.5e20        # reference mantle viscosity [Pa.s]
viscosity_max                       = 1.0e23        # maximum viscosity during simulation [Pa.s]
viscosity_min                       = 3.5e19        # minimum viscosity during simulation [Pa.s]
viscosity_per_element               = constant      # sets if viscosity is constant or linearly variable for every element, default is variable [constant/variable]
viscosity_mean_method               = arithmetic    # defines method do calculate the viscosity for each element, default is harmonic [harmonic/arithmetic]
viscosity_dependence                = pressure      # defines if viscosity depends on pressure or depth, default is depth [pressure/depth]

# External ASCII inputs/outputs
interfaces_from_ascii               = True          # set if interfaces between lithologies are read from an ASCII file (interfaces.txt), default is False [True/False]
n_interfaces                        = {len(interfaces.keys())}             # set the number of interfaces to be read from the interfaces ASCII file (interfaces.txt) 
temperature_from_ascii              = True          # set if initial temperature is read from an ASCII file (input_temperature_0.txt), default is False [True/False]
velocity_from_ascii                 = False         # set if initial velocity field is read from an ASCII file (input_velocity_0.txt), default is False [True/False]
variable_bcv                        = False         # allows velocity field re-scaling through time according to an ASCII file (scale_bcv.txt), default is False [True/False]
multi_velocity                      = False         # set if boundary velocities can change with time from ASCII file(s) (multi_veloc.txt and additional input_velocity_[X].txt files), default is False [True/False]
binary_output                       = False         # set if output is in binary format, default is False [True/False]
print_step_files                    = True          # set if the particles position are printed to an output file, default is True [True/False]

# Physical parameters
temperature_difference              = {temperature.values.max() - temperature.values.min()}            # temperature difference between the top and bottom of the model (relevant if <temperature_from_ascii> is False) [K]
thermal_expansion_coefficient       = {alpha}       # value for the coefficient of thermal expansion [1/K]
thermal_diffusivity_coefficient     = {kappa}        # value for the coefficient of thermal diffusivity [m^2/s]
gravity_acceleration                = {g}          # value for the gravity acceleration [m/s^2]
density_mantle                      = 3230.         # value for the mantle reference density [kg/m^3]
heat_capacity                       = {heat_capacity}         # value for the heat capacity [J/K] 
non_linear_method                   = on            # set if non linear method is used for the momentum equation [on/off]
adiabatic_component                 = off           # set if adiabatic heating/cooling is active [on/off]
radiogenic_component                = off           # set if radiogenic heating is active [on/off]

# Velocity boundary conditions
top_normal_velocity                 = fixed         # set the normal velocity on the top side of the model to be fixed or free [fixed/free] 
top_tangential_velocity             = free          # set the tangential velocity on the top side of the model to be fixed or free [fixed/free]
bot_normal_velocity                 = fixed         # set the normal velocity on the bottom side of the model to be fixed or free [fixed/free]
bot_tangential_velocity             = free          # set the tangential velocity on the bot side of the model to be fixed or free [fixed/free]
left_normal_velocity                = fixed         # set the normal velocity on the left side of the model to be fixed or free [fixed/free]
left_tangential_velocity            = free          # set the tangential velocity on the left side of the model to be fixed or free [fixed/free]
right_normal_velocity               = fixed         # set the normal velocity on the right side of the model to be fixed or free [fixed/free]
right_tangential_velocity           = free          # set the tangential velocity on the right side of the model to be fixed or free [fixed/free]

# Temperature boundary conditions
top_temperature                     = fixed         # set temperature on the top side of the model to be fixed or free [fixed/free]
bot_temperature                     = fixed         # set temperature on the bottom side of the model to be fixed or free [fixed/free]
left_temperature                    = free          # set temperature on the left side of the model to be fixed or free [fixed/free]
right_temperature                   = free          # set temperature on the right side of the model to be fixed or free [fixed/free]
rheology_model                      = 19             # flag n. of a pre-defined rheology model to use during simulation
T_initial                           = 0             # flag n. of a pre-defined temperature model to use during simulation (relevant when <temperature_from_ascii> is False)
"""

# Create the parameter file
with open(os.path.join(run_dir,"param.txt"), "w") as f:
    for line in params.split("\n"):
        f.write(line + "\n")